# Schrödinger equation for Hydrogen

We intend to solve the Schrödinger equation for the Hydrogen atom
\begin{equation*}
\left[\hat{T} + \hat{V}\right]\phi = E\phi
\end{equation*}
without resorting to the kinetic energy operator
\begin{equation*}
\hat{T} = -\frac{1}{2}\nabla^2
\end{equation*}


The potential operator contains only the nuclear attraction
\begin{equation*}
\hat{V}(r) = \frac{-Z}{|R-r|}
\end{equation*}
where $Z=1$ and $R=(0,0,0)$ are the nuclear charge and position, respectively. We iterate the Schrödinger equation in its integral form
\begin{equation*}
\tilde{\phi}^{n+1} = -2 \hat{H}^{\mu^n}[\hat{V} \phi^n]
\end{equation*}
where $\hat{H}^\mu$ is the integral convolution operator with the bound-state Helmholtz kernel $H^{\mu} = e^{-\mu r}/r$ and $\mu^n = \sqrt{-2E^n}$ is a positive real number.


The energy at iteration $n+1$ is computed as an update to the previous guess
\begin{equation*}
E^{n+1} = E^n + \frac{\langle\tilde{\phi}^{n+1} | \hat{V} | \tilde{\phi}^{n+1} - \phi^n \rangle}{\langle \tilde{\phi}^{n+1} | \tilde{\phi}^{n+1} \rangle}
\end{equation*}
and the wavefunction is normalized between each iteration $\phi^{n+1} = \tilde{\phi}^{n+1}/||\tilde{\phi}^{n+1}||^2$.

In [ ]:
from vampyr import vampyr3d as vp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Analytic nuclear potential: f_nuc(r) = Z/|r|
def f_nuc(r):
    Z = 1.0                 # Nuclear charge
    R = np.sqrt(r[0]*r[0] + r[1]*r[1] + r[2]*r[2])
    return -Z / R

# Analytic guess for wavefunction: f_phi(r) = exp(-r^2)
def f_phi(r):
    R2 = r[0]**2 + r[1]**2 + r[2]**2
    return np.exp(-R2)


# Set target precision
precision = 1.0e-4

# Define MRA basis and computational domain
k = 5                       # Polynomial order of MRA basis
world = [-20, 20]           # Computational domain [-L,L]^3 (a.u.)
MRA = vp.MultiResolutionAnalysis(order=k, box=world)

# Define projector onto the MRA basis
P_mra = vp.ScalingProjector(mra=MRA, prec=precision)

# Initialize the calculation
V = P_mra(f_nuc)            # Project analytic nuclear potential onto MRA
phi_n = P_mra(f_phi)        # Project analytic guess for wavefunction onto MRA
phi_n.normalize()           # Normalize the wavefunction guess
E_n = -1.0                  # Guess for energy (a.u.)
    
# Loop parameters
iter = 0                    # Iteration counter
thrs = 10*precision         # Set convergence threshold
update = 1.0                # Initialize error measure (norm of wavefunction update)
updates = []                # Will capture wavefunction updates for visusualization
energies = []               # Will capture energies for visualization

# Minimization loop
while (update > thrs):
    # Build Helmholtz operator from current energy
    mu = np.sqrt(-2*E_n)
    H = vp.HelmholtzOperator(mra=MRA, exp=mu, prec=precision)
    
    # Apply Helmholtz operator
    phi_np1 = -2*H(V*phi_n)
    
    # Compute wavefunction and energy update
    d_phi_n = phi_np1 - phi_n
    dE_n = vp.dot(phi_np1, V*d_phi_n) / phi_np1.squaredNorm()
    update = d_phi_n.norm()
    
    # Prepare for next iteration
    E_n += dE_n
    phi_n = phi_np1
    phi_n.normalize()

    # Collect output
    energies.append(E_n)
    updates.append(update)
    print(iter, " |  E:", E_n, " |  d_phi:", update)
    iter += 1


In [ ]:
updates_df = pd.DataFrame({"$\phi$":updates})
updates_df["$|\Delta \phi|$"] = updates_df.diff().abs()
updates_df.plot(y="$|\Delta \phi|$", logy=True, style=".-")
plt.title("Orbital convergence Hydrogen", fontstyle="italic", fontsize=12,fontweight='bold')
plt.xlabel("Iteration", fontstyle="italic", fontsize=12)
plt.ylabel("Orbital error", fontstyle="italic", fontsize=12)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.axhline(precision*10, linestyle="--", color="black", label="Threshod")
plt.legend()
plt.show()

In [ ]:
energies_df = pd.DataFrame({"$E$":energies})
energies_df["$\Delta E$"] = energies_df.diff().abs()
energies_df["$\Delta E$"].plot(logy=True, style=".-")
plt.title("Energy convergence Hydrogen", fontstyle="italic", fontsize=12,fontweight='bold')
plt.xlabel("Iteration", fontstyle="italic", fontsize=12)
plt.ylabel("Energy error", fontstyle="italic", fontsize=12)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.axhline(1.0e-8, linestyle="--", color="black", label="Precision")
plt.legend()
plt.savefig("energy_prec8.png", transparent=True, bbox_inches="tight")
plt.show()